In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
filename = '../data/alice-in-wonderland.txt'

s = Series(open(filename).read().split())
s.head()

0         ﻿The
1      Project
2    Gutenberg
3        EBook
4           of
dtype: object

# Beyond 1

What is the mean of all integers in Alice?

In [3]:
import string

(
    s
    .str
    .strip(string.punctuation)
    .loc[lambda s_: s_.str.isdigit()]
    .astype(int)
    .mean()
)

np.float64(8030.851851851852)

# Beyond 2

What words in Alice don't appear in the dictionary? Which are the five most common such words?

In [11]:
words = {one_word.strip().lower() for one_word in open('../data/words.txt', encoding='utf-8')}

In [12]:
(
    s
    .str.strip(string.punctuation)      # Strip punctuation
    .loc[lambda s_: s_.str.isalpha()]   # Keep only those with letters
    .loc[lambda s_: ~s_.isin(words)]    # Now keep those *not* in the dictionary, and find the most common ones
    .value_counts()
)

Alice            166
I                127
Project           83
The               63
She               36
                ... 
accepting          1
donors             1
offers             1
International      1
statements         1
Name: count, Length: 795, dtype: int64

In [13]:
(
    s
    .str.lower()                       # Convert to lowercase
    .str.strip(string.punctuation)      # Strip punctuation
    .loc[lambda s_: s_.str.isalpha()]   # Keep only those with letters
    .loc[lambda s_: ~s_.isin(words)]    # Now keep those *not* in the dictionary, and find the most common ones
    .value_counts()
)

gutenberg    30
terms        21
began        19
looked       18
heard        17
             ..
accepting     1
paperwork     1
takes         1
storyland     1
includes      1
Name: count, Length: 408, dtype: int64

# Beyond 3

What is the mean number of words per paragraph?

In [15]:
# Read the file into a series by paragraph
s = Series(open(filename, encoding='utf-8').read().split('\n\n'))

# Just use describe to get min, max, and everything else
(
    s
    .str
    .split()
    .str
    .len()
    .describe() 
)

count    393.000000
mean      32.475827
std       32.428415
min        0.000000
25%        7.000000
50%       22.000000
75%       48.000000
max      169.000000
dtype: float64